In [1]:
pip install dwave-qbsolv

In [2]:
!pip install -Uqq ipdb

     |████████████████████████████████| 792 kB 4.2 MB/s 
     |████████████████████████████████| 375 kB 52.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.26 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.1 which is incompatible.


1 Construct graph

In [3]:
class Node:
    
    def __init__(self, nodeID):
        self.__nodeID = nodeID
        self.__edges = []
    
    def getID(self):
        return self.__nodeID
    
    def getEdges(self):
        return self.__edges
    
    def hasEdge(self, edge):
        for i in range(0, len(self.__edges)):
            if (self.__edges[i].getID() == edge.getID()):
                return True
        return False
     
    def addEdge(self, edge):
        if (self.hasEdge(edge)):
            return False
        self.__edges.append(edge)
        
        edge.addNode(self)
        return True
    
    def equals(self, node):
        if (node.getID() is self.getID()):
            return True
        return False
    
    def getEdgeWithNode(self, node):
        for i in range(0, len(self.__edges)):
            if (self.__edges[i].getOtherNode(self).equals(node)):
                return self.__edges[i]
        return None


class Edge:
    
    def __init__(self, edgeID, nodeA = None, nodeB = None):
        self.__edgeID = edgeID
        self.__nodeA = nodeA
        self.__nodeB = nodeB
        if not (nodeA is None):
            nodeA.addEdge(self)
        if not (nodeB is None):
            nodeB.addEdge(self)
    
    def getOtherNode(self, node):
        if (self.__nodeA is node):
            return self.__nodeB
        elif (self.__nodeB is node):
            return self.__nodeA
        return None
    
    def getID(self):
        return self.__edgeID
    
    def getNodes(self):
        return (self.__nodeA, self.__nodeB)
    
    def addNode(self, node):
        if (self.__nodeA is None):
            self.__nodeA = node
            node.addEdge(self)
            return True
        elif (self.__nodeB is None and not self.__nodeA.getID() == node.getID()):
            self.__nodeB = node
            node.addEdge(self)
            return True
        return False
    
    def equals(self, edge):
        if (edge.getID() is self.getID()):
            return True
        return False

class Graph:
    
    def __init__(self):
        self.__nodes = []
        self.__edges = []
    
    def addNode(self, node):
        self.__nodes.append(node)
    
    def addNodes(self, nodes):
        self.__nodes.extend(nodes)
    
    def addEdge(self, edge):
        self.__edges.append(edge)
    
    def addEdges(self, edges):
        self.__edges.extend(edges)
    
    def getNodes(self):
        return self.__nodes
    
    def getEdges(self):
        return self.__edges
    
    def getNodeWithID(self, nodeID):
        for i in range(0, len(self.__nodes)):
            #print(str(i))
            '''if i == 621:
                print("test=" + str(self.__nodes[i]) + "getID=" + str(self.__nodes[i].getID()))'''
            if (self.__nodes[i].getID() == nodeID):
                return self.__nodes[i]
        return None
    
    def getEdgeWithID(self, edgeID):
        for i in range(0, len(self.__edges)):
            if (self.__edges[i].getID() is edgeID):
                return self.__edges[i]
        return None
    
    

def isRoadInRoute(road, route2):
    for i in range(0, len(route2)):
        if (road.equals(route2[i])):
            return True
    return False

def getIntersections(route1, route2):
    intersections = []
    for i in range(0, len(route1)):
        if (route1[i] in route2): #isRoadInRoute(route1[i], route2)):
            intersections.append(route1[i])
    return intersections

def getUniqueArr(arr):
    ansArr = []
    for i in range(0, len(arr)):
        if(arr[i] not in ansArr):
            ansArr.append(arr[i])
    return ansArr

2 Data sanitization

In [4]:
import os
import ipdb
import networkx as nx
import matplotlib.pyplot as plt
import sys
from dwave_qbsolv import QBSolv
from IPython.core.debugger import Tracer

graph_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/example_189.txt')
file = open(graph_path, 'r')
lines = file.readlines()
file.close()
    
#delete any comments and empty lines
for i in range(len(lines)-1, -1, -1):
    #delete the newline character at the end
    temp = lines[i].split("\n")
    lines[i] = temp[0]
        
    temp = lines[i].split(" #")
    lines[i] = temp[0]
    temp = lines[i].split("#")
    lines[i] = temp[0]
        
    if (lines[i] is ""):
        tmp = lines.pop(i)
    
numOfNodes = int(lines[0])
#initialize graph and nodes
graph = Graph()
for i in range(0, numOfNodes):
    node = Node(i)
    graph.addNode(node)

%debug    
#initialize edges
nextLine = 1
numOfEdges = 0
for i in range(0, numOfNodes):

    nodeIDs = lines[nextLine+i].split(" ")

    for j in range(0, len(nodeIDs)):

        nodeA = graph.getNodeWithID(i)
        nodeB = graph.getNodeWithID(int(nodeIDs[j]))
        edge_existing = nodeA.getEdgeWithNode(nodeB)
        if not (edge_existing is None):
            continue
        edge_existing = nodeB.getEdgeWithNode(nodeA)
        if not (edge_existing is None):
            continue
        
        #Tracer()()
        
        print("i="+str(i)+" j="+str(j)+"  nodeA="+str(nodeA.getID()) + " nodeB="+str(nodeB.getID())+ "numOfEdges=" + str(numOfEdges))
        
        edge = Edge(numOfEdges, nodeA, nodeB)
        
        graph.addEdge(edge)
  
        numOfEdges = numOfEdges + 1

nextLine = nextLine + numOfNodes
numOfCars = int(lines[nextLine])
nextLine = nextLine + 1


ERROR:root:No traceback has been produced, nothing to debug.


Streaming output truncated to the last 5000 lines.
i=42912 j=3  nodeA=42912 nodeB=42914numOfEdges=91865
i=42913 j=3  nodeA=42913 nodeB=42914numOfEdges=91866
i=42915 j=1  nodeA=42915 nodeB=42927numOfEdges=91867
i=42916 j=1  nodeA=42916 nodeB=42927numOfEdges=91868
i=42917 j=1  nodeA=42917 nodeB=42927numOfEdges=91869
i=42919 j=2  nodeA=42919 nodeB=42920numOfEdges=91870
i=42920 j=4  nodeA=42920 nodeB=42939numOfEdges=91871
i=42920 j=5  nodeA=42920 nodeB=42940numOfEdges=91872
i=42920 j=6  nodeA=42920 nodeB=48111numOfEdges=91873
i=42922 j=4  nodeA=42922 nodeB=47366numOfEdges=91874
i=42927 j=6  nodeA=42927 nodeB=42932numOfEdges=91875
i=42928 j=1  nodeA=42928 nodeB=42929numOfEdges=91876
i=42930 j=2  nodeA=42930 nodeB=42931numOfEdges=91877
i=42932 j=4  nodeA=42932 nodeB=47364numOfEdges=91878
i=42933 j=1  nodeA=42933 nodeB=48226numOfEdges=91879
i=42937 j=2  nodeA=42937 nodeB=47364numOfEdges=91880
i=42938 j=8  nodeA=42938 nodeB=47391numOfEdges=91881
i=42939 j=3  nodeA=42939 nodeB=47366numOfEdges=9

3 Get car routes

In [8]:

#get original car routes for each car
graph_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/example_174.txt')
file = open(graph_path, 'r')
lines = file.readlines()
file.close()
#delete any comments and empty lines
for i in range(len(lines)-1, -1, -1):
    #delete the newline character at the end
    temp = lines[i].split("\n")
    lines[i] = temp[0]
        
    temp = lines[i].split(" #")
    lines[i] = temp[0]
    temp = lines[i].split("#")
    lines[i] = temp[0]
        
    if (lines[i] is ""):
        tmp = lines.pop(i)


carRoutes = [] 
numOfCars = 174  
nextLine = 49551 
for i in range(0, numOfCars):
    carRoutes.append([])
    
for i in range(0, numOfCars):
    nodeIDs = lines[nextLine+i].split("-")
    route = []
    print("i="+str(lines[nextLine+i]))
    for j in range(0, len(nodeIDs)-1):
        print("j="+str(j))
        nodeA = graph.getNodeWithID(int(nodeIDs[j]))
        nodeB = graph.getNodeWithID(int(nodeIDs[j+1]))
        edge = nodeA.getEdgeWithNode(nodeB)
        print("i="+str(i)+" j="+str(j)+"  nodeA="+str(nodeA.getID()) + " nodeB="+str(nodeB.getID())+ "edge=" + str(edge))
        route.append(edge)
    carRoutes[i].append(route)
nextLine = nextLine + numOfCars
  
#get the alternate routes
numOfAlternateRoutes = 3
for i in range(0, numOfCars*(numOfAlternateRoutes-1)):
    carNum = int(lines[nextLine+i].split(" ")[0])
    nodeIDs = (lines[nextLine+i].split(" ")[1]).split("-")
    route = []
    for j in range(0, len(nodeIDs)-1):
        nodeA = graph.getNodeWithID(int(nodeIDs[j]))
        nodeB = graph.getNodeWithID(int(nodeIDs[j+1]))
        edge = nodeA.getEdgeWithNode(nodeB)
        route.append(edge)
    print("route="+ str(route))
    carRoutes[carNum].append(route)


Streaming output truncated to the last 5000 lines.
i=164 j=105  nodeA=21909 nodeB=21909edge=None
j=106
i=164 j=106  nodeA=21909 nodeB=21909edge=None
j=107
i=164 j=107  nodeA=21909 nodeB=21909edge=None
j=108
i=164 j=108  nodeA=21909 nodeB=21909edge=None
j=109
i=164 j=109  nodeA=21909 nodeB=21909edge=None
j=110
i=164 j=110  nodeA=21909 nodeB=21909edge=None
j=111
i=164 j=111  nodeA=21909 nodeB=21909edge=None
j=112
i=164 j=112  nodeA=21909 nodeB=21909edge=None
j=113
i=164 j=113  nodeA=21909 nodeB=21935edge=<__main__.Edge object at 0x7f44aa7277d0>
j=114
i=164 j=114  nodeA=21935 nodeB=21912edge=None
j=115
i=164 j=115  nodeA=21912 nodeB=21909edge=None
j=116
i=164 j=116  nodeA=21909 nodeB=21935edge=<__main__.Edge object at 0x7f44aa7277d0>
j=117
i=164 j=117  nodeA=21935 nodeB=21935edge=None
j=118
i=164 j=118  nodeA=21935 nodeB=21935edge=None
j=119
i=164 j=119  nodeA=21935 nodeB=21935edge=None
j=120
i=164 j=120  nodeA=21935 nodeB=21935edge=None
j=121
i=164 j=121  nodeA=21935 nodeB=21935edge=None

4 Run on quantum computer

In [9]:
def getQuboMatrix(carRoutes, numOfAlternateRoutes):
    #getting the Q (qubo matrix) from carRoutes array (list)
    Q = {}
    for i in range(0, len(carRoutes)*numOfAlternateRoutes): # number of cars * number of alternate routes
        for j in range(0, len(carRoutes)*numOfAlternateRoutes):
            Q.update({(i, j): 0})

    carRouteIntersections = [0] * len(carRoutes)
    for i in range(0, len(carRoutes)):
        for j in range(0, numOfAlternateRoutes):
            # get the number of total intersections and write to this array
            carIntersections = []
            for k in range(i+1, len(carRoutes)):
                for m in range(0, numOfAlternateRoutes):
                    # get intersections of the two routes (not the same car)
                    intersections = getIntersections(carRoutes[i][j], carRoutes[k][m])
                    # write to the upper triangle
                    Q.update({(i*numOfAlternateRoutes+j, k*numOfAlternateRoutes+m): len(intersections)*2})
                    # save to the total intersections for diagonals
                    carIntersections.extend(intersections)
            # write to the diagonals
            Q.update({(i*numOfAlternateRoutes+j, i*numOfAlternateRoutes+j): len(carIntersections)})
            # keep the number of unique intersections for lambda
            carRouteIntersections[i] = carRouteIntersections[i] + len(getUniqueArr(carIntersections))

    lam = max(carRouteIntersections)
    for i in range(0, len(carRoutes)):
        for j in range(0, numOfAlternateRoutes):
            # write "2*lambda" to the upper small triangle of each vehicle
            for k in range(j+1, numOfAlternateRoutes):
                Q.update({(i*numOfAlternateRoutes+j, i*numOfAlternateRoutes+k): lam*2})
            # subtract lambda from the diagonals
            Q.update({(i*numOfAlternateRoutes+j, i*numOfAlternateRoutes+j): (Q[(i*numOfAlternateRoutes+j, i*numOfAlternateRoutes+j)] - lam)})
    return Q


Q = getQuboMatrix(carRoutes, 3)
    
response = QBSolv().sample_qubo(Q)
result = list(response.samples())
result = result[0]
resultingEnergy = list(response.data_vectors['energy'])
print("list of energies=" + str(resultingEnergy))
resultingEnergy = resultingEnergy[0]

print("samples=" + str(result))
print("Resulting energy: " + str(resultingEnergy))

print("len carRoutes:" + str(len(carRoutes)))
final_routes = []
for i in range(0, 174):
        final_routes.append([])
for i in range(0, len(result)):
      if (result[i] == 1):
          print("i by numOfAlternateRoutes=" + str(i//3))
          print("(i rest numOfAlternateRoutes) = " +str((i%3)))
          kharejGh = carRoutes[(i//3)]
          restTest = kharejGh[(i%3)]
          print("kharej=" + str(kharejGh))
          print("rest = " +str(restTest))
          print("******:" + str(carRoutes[(i//3)][(i%3)]))
          final_routes[(i//3)] = carRoutes[(i//3)][(i%3)]
          print("car " + str((i//3)+1) + ": route " + str((i%3)+1))
print("final_routes:" + str(final_routes))
array_trj_all_cars = []
for mm in range(0, len(final_routes)):
    array_trj_each_car = []
    for ee in range(0, len(final_routes[mm])):
            print("final_routes mm:" + str(final_routes[mm]))
            nodeA = final_routes[mm][ee].getNodes()[0].getID()
            array_trj_each_car.append(nodeA)
            nodeB = final_routes[mm][ee].getNodes()[1].getID()
            array_trj_each_car.append(nodeB)
    array_trj_all_cars.append(array_trj_each_car)
    print("test" + str(array_trj_all_cars))


with open('/content/drive/MyDrive/Colab Notebooks/QC_code/ISPRS/results.txt', 'a') as f:
        f.truncate(0)        
        f.write(str(array_trj_all_cars))  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: DeprecationWarning: QBSolv is deprecated as of the end of 2021 and support will be discontinued after March 2022. Please update your code to use dwave-hybrid or Leap's quantum-classical hybrid solvers instead.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

